# Deploy Vespa apps to Docker

> Python API to deploy Vespa applications to Docker containers.

This tutorial illustrate how to deploy a Vespa application to a Docker container in your local machine. It is required to have Docker installed in the machine you are running this tutorial from. For that reason we cannot run this tutorial in Google Colab as Docker is not available on their standard runtime machines.

## Application package API

We first create a `Document` instance containing the `Field`s that we want to store in the app. In this case we will keep the application simple and only feed a unique `id`, `title` and `body` of the MS MARCO documents.

In [1]:
from vespa.package import Document, Field

document = Document(
    fields=[
        Field(name = "id", type = "string", indexing = ["attribute", "summary"]),
        Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
        Field(name = "body", type = "string", indexing = ["index", "summary"], index = "enable-bm25")        
    ]
)

The complete `Schema` of our application will be named `msmarco` and contains the `Document` instance that we defined above, the default `FieldSet` indicates that queries will look for matches by searching both in the titles and bodies of the documents. The default `RankProfile` indicates that all the matched documents will be ranked by the `nativeRank` expression involving the title and the body of the matched documents.

In [2]:
from vespa.package import Schema, FieldSet, RankProfile

msmarco_schema = Schema(
    name = "msmarco", 
    document = document, 
    fieldsets = [FieldSet(name = "default", fields = ["title", "body"])],
    rank_profiles = [RankProfile(name = "default", first_phase = "nativeRank(title, body)")]
)

Once the `Schema` is defined, all we have to do is to create our msmarco `ApplicationPackage`:

In [3]:
from vespa.package import ApplicationPackage

app_package = ApplicationPackage(name = "msmarco", schema=msmarco_schema)

At this point, `app_package` contains all the relevant information required to create our MS MARCO text search app. We now need to deploy it.

## Deploy to a Docker container

For the following to work you need to run this from a machine with Docker installed. We first create a `VespaDocker` instance based on the application package.

In [4]:
from vespa.package import VespaDocker

vespa_docker = VespaDocker(port=8089)

We then call the `deploy` method and specify a `disk_folder`. Behind the scenes, `pyvespa` will write the Vespa config files and store them in the `disk_folder`, it will then run a Vespa engine Docker container and deploy those config files in the container.

In [ ]:
import os

disk_folder = os.path.join(os.getenv("WORK_DIR"), "sample_application")

app = vespa_docker.deploy(
    application_package = app_package,
    disk_folder=disk_folder
)

The app variable above will hold a `Vespa` instance that will be used to connect and interact with our text search application. We can see the deployment message returned by the Vespa engine:

In [ ]:
app.deployment_message

## Interact with the deployed application

From this point on you can interact with the deployed application the same way we did in the following tutorials:
* [How to connect with running Vespa instances](connect-to-vespa-instance.ipynb)
* [Build end-to-end Vespa apps and deploy to Vespa Cloud](create-and-deploy-vespa-cloud.ipynb)

## Clean up environment

In [8]:
from shutil import rmtree

rmtree(disk_folder, ignore_errors=True)
vespa_docker.container.stop()
vespa_docker.container.remove()